In [1]:
import ckanapi
from ckanapi.errors import NotFound,CKANAPIError, NotAuthorized
import pandas as pd
import re
from requests_cache  import CachedSession

In [2]:
session = CachedSession(
    
)
try:
    ua_portal = ckanapi.RemoteCKAN('https://data.carpathia.gov.ua/')
    datasetTwo = ua_portal.action.package_show(id='f9a3dd3a-0204-490d-b6e0-013ddecfcc4c')
    print(f'Title: {datasetTwo['title']}')

    resources_url = None
    for rec in datasetTwo['resources']:
        
        if rec['format'].lower() == 'geojson':
            resources_url = rec['url']
            break

    if resources_url:
        response = session.get(resources_url)
        response.raise_for_status()

        geo_data =  response.json()  
        df_bombshelter = pd.json_normalize(geo_data['features'])         
except NotFound:
    print("Error: ID of dataset doesn`t exist on this site;")
except NotAuthorized:
    print("Error: This dataset is private or requires API key to access;")
except CKANAPIError as e:
    print(f"Error:A CKAN API error occured: {e}; ")
except Exception as e:
    print(f"An unexpected error ocurred: {e};")

Title: Адміністративні дані, що збираються (обробляються) та підлягають оприлюдненню управлінням цивільного захисту


In [13]:
s_bezbar = df_bombshelter["properties.Bezbar"].value_counts(normalize=True)*100
print(f"${s_bezbar.loc["true"]:,.2f}%")
s_bezbar

$16.22%


properties.Bezbar
false    83.527454
true     16.222962
          0.249584
Name: proportion, dtype: float64

In [5]:
def clean_num(s:pd.Series) -> pd.Series:
    s_Num = s.astype(dtype=str)
    s_Num = s_Num.str.replace(r'\s+', '', regex=True)
    s_Num = s_Num.str.replace(r'[А-Яа-яA-za-z]', '', regex=True) 
    
    regexQuotes = r'[“”„\?»«]'
    s_Num = s_Num.str.replace(regexQuotes, '', regex=True)

    s_Num = s_Num.str.replace(',' , '.') 
    
    s_Num = s_Num.str.strip()

    return pd.to_numeric(s_Num)

In [6]:
coords_df = pd.DataFrame(df_bombshelter['geometry.coordinates'].to_list(), index = df_bombshelter.index)

coords_df[0] = clean_num(coords_df[0])
coords_df[1] = clean_num(coords_df[1])
    
coords_df = coords_df.rename(columns={ 0:'longitude', 1:'latitude'})
display(coords_df)
display(df_bombshelter['geometry.coordinates'])

,longitude,latitude
0,22.743177,48.445444
1,23.574922,48.014598
2,22.748696,48.445722
3,22.306102,48.600162
4,22.719559,48.439289
...,...,...
1197,22.998404,48.413013
1198,23.206663,48.727302
1199,22.876351,48.876656
1200,22.870597,48.526744


0       [22.743177, 48.445444]
1       [23.574922, 48.014598]
2       [22.748696, 48.445722]
3       [22.306102, 48.600162]
4       [22.719559, 48.439289]
                 ...          
1197    [22.998404, 48.413013]
1198    [23.206663, 48.727302]
1199    [22.876351, 48.876656]
1200    [22.870597, 48.526744]
1201    [24.000164, 47.971012]
Name: geometry.coordinates, Length: 1202, dtype: object

In [13]:
if coords_df['longitude'].dtype == float:
    print('yes')
else:
    print(coords_df['longitude'].dtype)

yes


In [7]:
string = 'properties.OTG'
strFalse = 'preporties.OTG'
if string  not in df_bombshelter.columns:
    print("yes")
else:
    print("No") 
display(df_bombshelter.columns)
#print(df_bombshelter.iloc[1067])
#display(df_bombshelter["properties.Property"].drop_duplicates())


No


Index(['type', 'properties.OTG', 'properties.Number', 'properties.City',
       'properties.Name', 'properties.Area', 'properties.People',
       'properties.TypeZs', 'properties.Property', 'properties.Adress',
       'properties.Rajon', 'properties.Type', 'properties.Bezbar',
       'geometry.type', 'geometry.coordinates'],
      dtype='object')

In [1]:
from folium.plugins import MarkerCluster

In [5]:
display(df_bombshelter.iloc[325])

type                                       Feature
properties.OTG                         Кольчинська
properties.Number                                0
properties.City                           с. Крите
properties.Name         Критенська початкова школа
properties.Area                                 28
properties.People                               43
properties.TypeZs                        вбудоване
properties.Property                     Комунальна
properties.Adress                              50А
properties.Rajon                      Мукачівський
properties.Type                Найпростіші укриття
properties.Bezbar                            false
geometry.type                                Point
geometry.coordinates          [22.698368, 48.6091]
Name: 325, dtype: object

In [4]:
df_otg = df_bombshelter["properties.OTG"].copy().astype(str)
df_otg = df_otg.astype(str).str.replace(r'[\n\t]', '', regex=True)
df_otg = df_otg.str.replace('`',"'").str.replace("’", "'")
df_otg = df_otg.str.replace(r'\s*-\s*', '-', regex=True)
combined_regex = r'(?:\s+(?:ТГ|ОТГ|тг|отг|СТГ|стг).*$)'
df_otg = df_otg.str.replace(combined_regex, '', regex=True, flags=re.IGNORECASE)
df_otg = df_otg.str.replace(r'\s+[А-Яа-яA-za-z]\b$', '', regex=True)
df_otg = df_otg.str.replace(r'[\d\",]', '', regex=True)
df_otg = df_otg.str.replace(r'\.$|^\.', '', regex=True)
fixed_dctionary = {
    'Усть-Чорна':'Усть-Чорнянська',
    'Косонська' : 'Косоньська',
}
df_otg = df_otg.replace(fixed_dctionary)
df_otg = df_otg.str.strip()
df_otg = df_otg.drop_duplicates()
def ukr_sort_key(text):
    ukr_alphabet = "АБВГҐДЕЄЖЗИІЇЙКЛМНОПРСТУФХЦЧШЩЬЮЯ"
    sort_map = {c: i for i, c in enumerate(ukr_alphabet)}
    return [sort_map.get(c.upper(), 999) for c in str(text)]


df_sorted = df_otg.sort_values(key=lambda col: col.map(ukr_sort_key))
display(df_sorted)


1144     Баранинська
324        Батівська
404     Бедевлянська
33       Берегівська
184       Білківська
            ...     
8        Холмківська
31          Хустська
166     Чинадіївська
59           Чопська
55        Ясінянська
Name: properties.OTG, Length: 61, dtype: object

In [ ]:
df_categorized = df_bombshelter[(df_bombshelter["properties.City"] == "м. Мукачево")]
display(df_categorized)


,type,properties.OTG,properties.Number,properties.City,properties.Name,properties.Area,properties.People,properties.TypeZs,properties.Property,properties.Adress,properties.Rajon,properties.Type,properties.Bezbar,geometry.type,geometry.coordinates
2,Feature,Мукачівська,0,м. Мукачево,“Зі-Північ”,90,150,вбудоване,Приватна,"вул.Свалявська,78",Мукачівський,Споруди подвійного призначення,false,Point,"[22.748696, 48.445722]"
4,Feature,Мукачівська,24416,м. Мукачево,"ТОВ ""Захід Фінанс Груп""",51.3,88,вбудоване,Приватна,"вул. Я. Мудрого, 10",Мукачівський,Сховище,false,Point,"[22.719559, 48.439289]"
5,Feature,Мукачівська,24425,м. Мукачево,ЗЗСО №4,84.4,124,вбудоване,Комунальна,"вул. Ілони Зріні, 34",Мукачівський,Сховище,false,Point,"[22.713257, 48.439896]"
6,Feature,Мукачівська,24409,м. Мукачево,Житловий будинок УМГ,92,184,вбудоване,Комунальна,"вул. Ілони Зріні, 10",Мукачівський,Сховище,false,Point,"[22.715701, 48.440023]"
7,Feature,Мукачівська,24424,м. Мукачево,ЗЗСО № 1,324,150,вбудоване,Комунальна,"вул. Соборна, 23",Мукачівський,Сховище,false,Point,"[22.717655, 48.438853]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,Feature,Мукачівська,0,м. Мукачево,Мукачівський Державний університет (корпус 3),30,51,вбудоване,Державна,"пл.Федорова,5",Мукачівський,Найпростіші укриття,false,Point,"[22.719263, 48.440181]"
1131,Feature,Мукачівська,0,м. Мукачево,ЗДО №2,340,566,вбудоване,Комунальна,"вул.Водна,13а",Мукачівський,Найпростіші укриття,false,Point,"[22.710077, 48.445057]"
1166,Feature,Мукачівська,0,м. Мукачево,"Приватна гімназія ""Еммануїл""",54,90,вбудоване,Приватна,"вул. Набережна Незалежності (Мічуріна), 3",Мукачівський,Найпростіші укриття,false,Point,"[22.70531, 48.442356]"
1169,Feature,Мукачівська,0,м. Мукачево,Гуртожиток Мукачівського ліцею ім.Святого Ішт...,114,189,вбудоване,Приватна,"вул.Миру, 15",Мукачівський,Найпростіші укриття,false,Point,"[22.723987, 48.439871]"


In [4]:
df_cityName = df_bombshelter["properties.City"].drop_duplicates() 
display(df_cityName)

0          м.Мукачево,
1              м.Тячів
2          м. Мукачево
3            м.Ужгород
8           с. Розівка
             ...      
1183       с. Петрушів
1184           с.Есень
1189        с. Павлово
1196    с. Чорноголова
1199       с. Кічерний
Name: properties.City, Length: 407, dtype: object

In [ ]:
df_norm_city = df_bombshelter["properties.City"].copy()
display(df_norm_city)

0           м.Мукачево,
1               м.Тячів
2           м. Мукачево
3             м.Ужгород
4           м. Мукачево
             ...       
1197    с. Чорний Потік
1198          с. Канора
1199        с. Кічерний
1200         с. Карпати
1201    смт Вел. Бичків
Name: properties.City, Length: 1202, dtype: object

In [ ]:
display(df_norm_city)
df_norm_city = df_norm_city.str.replace(r'^[а-я. ]*|[,]|[0-9]|[ ]$|[.]$|^[\"]$', "",  regex=True)
df_norm_city = df_norm_city.str.replace('I', 'І')
df_norm_city = df_norm_city.str.replace('i', 'і')
df_norm_city = df_norm_city.str.replace('Березово', 'Березове').drop_duplicates()

display(df_norm_city.sort_values(ascending=True))

0           м.Мукачево,
1               м.Тячів
2           м. Мукачево
3             м.Ужгород
4           м. Мукачево
             ...       
1197    с. Чорний Потік
1198          с. Канора
1199        с. Кічерний
1200         с. Карпати
1201    смт Вел. Бичків
Name: properties.City, Length: 1202, dtype: object

342      Іванівці
246           Іза
241          Ізки
392       Ільниця
312     Імстичово
          ...    
394      Шишлівці
427    Щасливе  Б
472     Щербовець
997      Яблуново
55          Ясіня
Name: properties.City, Length: 357, dtype: object

In [13]:
import re
s = df_bombshelter["properties.City"].copy()
homoglyphs = {
    'A': 'А', 'B': 'В', 'C': 'С', 'E': 'Е', 'H': 'Н', 'I': 'І', 'K': 'К', 
    'M': 'М', 'O': 'О', 'P': 'Р', 'T': 'Т', 'X': 'Х', 'i': 'і', 'y': 'у',
    'a': 'а', 'c': 'с', 'e': 'е', 'o': 'о', 'p': 'р', 'x': 'х'
    }

trans_table = str.maketrans(homoglyphs)
s = s.apply(lambda x: x.translate(trans_table))

prefix_pattern = r'^\s*(?:м\.|с\.|смт\.?|пос\.|місто|село|селище)\s*'
s = s.str.replace(prefix_pattern, "", regex=True, flags=re.IGNORECASE)

s = s.str.replace(r'[\d\",]', '', regex=True)
s = s.str.replace(r'\.$', '', regex=True)

s = s.drop_duplicates()

display(s)

0          Мукачево
1             Тячів
3           Ужгород
8           Розівка
15        Противень
           ...     
1180     Вільховиця
1183       Петрушів
1189        Павлово
1196    Чорноголова
1199       Кічерний
Name: properties.City, Length: 367, dtype: object

In [14]:
s = s.astype(str).str.replace(r'[\n\t]', '', regex=True)
s = s.str.replace('`',"'").str.replace("’", "'")
s = s.str.replace(r'\s*-\s*', '-', regex=True)
s = s.str.replace(r'\s+(вул\.|ул\.|пл\.|кв\.).*$', '', regex=True, flags=re.IGNORECASE)
s = s.str.replace(r'\s+[А-Яа-яA-za-z]\b$', '', regex=True)

abbreviation_map ={
        'Вел. Бичків': 'Великий Бичків',
        'В.Бичків': 'Великий Бичків',
        'В.Ворота': 'Верхні Ворота',   # Plural
        'Н.Ворота': 'Нижні Ворота',    # Plural
        'В.Коропець': 'Верхній Коропець', # Masculine
        'Н.Коропець': 'Нижній Коропець',  # Masculine
        'В.Визниця': 'Верхня Визниця',    # Feminine
        'М.Раковець': 'Малий Раковець',
        'В.Раковець': 'Великий Раковець',
        'Р.Поле': 'Руське Поле',
        'Н.Селище': 'Нижнє Селище',
        'В.Водяне': 'Верхнє Водяне'
}
s = s.replace(abbreviation_map)

typo_correct = {
        'Золотарево': 'Золотарьово',   # Standardizing on 'yo'
        'Зарічово': 'Зарічево',
        'Копашнево': 'Копашново',
        'Кленовець': 'Кленовець',
        'Клиновець': 'Кленовець',      # Fix typo
        'Верхне Водяне': 'Верхнє Водяне', # Fix 'e' -> 'є'
        'Горінчево': 'Горінчово',
        'Усть- Чорна': 'Усть-Чорна',
        'Бедевля': 'Бедевля',
}

s = s.replace(typo_correct)

In [15]:
def ukr_sort_key(text):
    ukr_alphabet = "АБВГҐДЕЄЖЗИІЇЙКЛМНОПРСТУФХЦЧШЩЬЮЯ"
    sort_map = {c: i for i, c in enumerate(ukr_alphabet)}
    return [sort_map.get(c.upper(), 999) for c in text] 

df_sorted = s.sort_values(key=lambda col: col.map(ukr_sort_key))
display(df_sorted)

723       Абранка
163     Анталовці
1150     Арданово
285         Астей
576        Бабичі
          ...    
427       Щасливе
472     Щербовець
997      Яблуново
55          Ясіня
79      .Кольчино
Name: properties.City, Length: 367, dtype: object

In [ ]:
import pandas as pd
import re

# Load your data (assuming df_bombshelter is already loaded)
# df_bombshelter = pd.read_csv('testCityName.csv') 

s = df_bombshelter["properties.City"].copy().astype(str)

# --- 1. Fix Homoglyphs (Latin -> Cyrillic) ---
homoglyphs = {
    'A': 'А', 'B': 'В', 'C': 'С', 'E': 'Е', 'H': 'Н', 'I': 'І', 'K': 'К', 
    'M': 'М', 'O': 'О', 'P': 'Р', 'T': 'Т', 'X': 'Х', 'i': 'і', 'y': 'у',
    'a': 'а', 'c': 'с', 'e': 'е', 'o': 'о', 'p': 'р', 'x': 'х'
}
trans_table = str.maketrans(homoglyphs)

# ERROR WAS HERE: You must assign 's ='
s = s.apply(lambda x: x.translate(trans_table))

# --- 2. Remove Prefixes ---
prefix_pattern = r'^\s*(?:м\.|с\.|смт\.?|пос\.|місто|село|селище)\s*'

# ERROR WAS HERE: You must assign 's ='
s = s.str.replace(prefix_pattern, "", regex=True, flags=re.IGNORECASE)

# --- 3. Clean Garbage ---
s = s.astype(str).str.replace(r'[\n\t]', '', regex=True)
s = s.str.replace('`',"'").str.replace("’", "'")
s = s.str.replace(r'\s*-\s*', '-', regex=True)
combined_regex = r'(?:\s+(?:вул\.|ул\.|пл\.|кв\.).*|вул)\s*$'
s = s.str.replace(combined_regex, '', regex=True, flags=re.IGNORECASE)
s = s.str.replace(r'(вул)$', '', regex=True)
s = s.str.replace(r'\s+[А-Яа-яA-za-z]\b$', '', regex=True)
s = s.str.replace(r'[\d\",]', '', regex=True)
s = s.str.replace(r'\.$|^\.', '', regex=True)
s = s.str.strip()

# --- 4. Abbreviations & Typos ---
abbreviation_map = {
    'Вел. Бичків': 'Великий Бичків',
    'В.Бичків': 'Великий Бичків',
    'В.Ворота': 'Верхні Ворота',
    'В. Ворота': 'Верхні Ворота',
    'Н.Ворота': 'Нижні Ворота',
    'В.Коропець': 'Верхній Коропець',
    'Н.Коропець': 'Нижній Коропець',
    'В.Визниця': 'Верхня Визниця',
    'М.Раковець': 'Малий Раковець',
    'В.Раковець': 'Великий Раковець',
    'Р.Поле': 'Руське Поле',
    'Н.Селище': 'Нижнє Селище',
    'В.Водяне': 'Верхнє Водяне',
    'Н.Давидково':'Нове Давидково', 
    'Н. Ремета' : 'Нижні Ремети',
    

}
s = s.replace(abbreviation_map)

typo_correct = {
    'Золотарево': 'Золотарьово',
    'Зарічово': 'Зарічево',
    'Копашнево': 'Копашново',
    'Кленовець': 'Кленовець',
    'Клиновець': 'Кленовець',
    'Верхне Водяне': 'Верхнє Водяне',
    'Горінчево': 'Горінчово',
    'Усть- Чорна': 'Усть-Чорна',
    'Бедевля': 'Бедевля',
    'Березово':'Березове',
    'Оклі' : 'Оклі Гедь',
    'Горинчево': 'Горінчово',
    'Вільхівські -Лази':'Вільхівські-Лази',
    'Оклі Гедь Гедь' : 'Оклі Гедь',
    'Неветленфолувул' : 'Неветленфолу'
}
s = s.replace(typo_correct, regex=True)

# --- 5. Sort with Ukrainian Alphabet ---
def ukr_sort_key(text):
    ukr_alphabet = "АБВГҐДЕЄЖЗИІЇЙКЛМНОПРСТУФХЦЧШЩЬЮЯ"
    sort_map = {c: i for i, c in enumerate(ukr_alphabet)}
    # Convert text to a list of ranks
    return [sort_map.get(c.upper(), 999) for c in str(text)]

s = s.drop_duplicates()
df_sorted = s.sort_values(key=lambda col: col.map(ukr_sort_key))

print("Cleaned and Sorted Data:")
display(df_sorted)

Cleaned and Sorted Data:


723       Абранка
163     Анталовці
1150     Арданово
285         Астей
576        Бабичі
          ...    
394      Шишлівці
427       Щасливе
472     Щербовець
997      Яблуново
55          Ясіня
Name: properties.City, Length: 337, dtype: object

In [5]:
df_Bezbar = df_bombshelter["properties.Bezbar"].copy()
bool_dict={'true' : True, 
           'false' : False}
df_Bezbar = df_Bezbar.replace(bool_dict)


In [12]:
display(df_Bezbar)
if type(df_Bezbar.iloc[0]) is bool:
    print("yes")
else:
    print("No")

0       False
1       False
2       False
3       False
4       False
        ...  
1197     True
1198     True
1199    False
1200    False
1201    False
Name: properties.Bezbar, Length: 1202, dtype: object

yes
